In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [12]:
data = pd.read_csv('/Users/Muyan/Desktop/ML_Group_Project/train.csv')

In [21]:
data['GarageFinish'].replace(to_replace = "Na","Unf","Rfn","Fin",
                            value = "0","1","2","3")

SyntaxError: positional argument follows keyword argument (<ipython-input-21-0a7c91b18c9f>, line 1)

In [31]:
data.GarageFinish = data.GarageFinish.replace('NaN','0',regex = True)

In [24]:
data.GarageFinish = data.GarageFinish.replace('Unf','1',regex = True)

In [29]:
data.GarageFinish = data.GarageFinish.replace('RFn','2',regex = True)

In [26]:
data.GarageFinish = data.GarageFinish.replace('Fin','3',regex = True)

In [33]:
# data.GarageFinish.sample(10)

In [19]:
type(data['GarageArea'])

pandas.core.series.Series

In [34]:
data.GarageQual = data.GarageQual.replace('EX','Exellent',regex = True)

In [35]:
data.GarageQual = data.GarageQual.replace('TA','Avarage',regex = True)

In [36]:
data.GarageQual = data.GarageQual.replace('Gd','Good',regex = True)

In [37]:
data.GarageQual = data.GarageQual.replace('Fa','Fair',regex = True)

In [38]:
data.GarageQual = data.GarageQual.replace('NA','no garage',regex = True)

In [48]:
# data.GarageQual.sample(10)

In [43]:
data.GarageCond = data.GarageCond.replace('EX','Exellent',regex = True)

In [44]:
data.GarageCond = data.GarageCond.replace('TA','Avarage',regex = True)

In [45]:
data.GarageCond = data.GarageCond.replace('Gd','Good',regex = True)

In [46]:
data.GarageCond = data.GarageCond.replace('Fa','Fair',regex = True)

In [47]:
data.GarageCond = data.GarageCond.replace('NA','no garage',regex = True)

In [51]:
# data.GarageCond.sample(10)

In [52]:
data.WoodDeckSF.isna().sum()

0

In [54]:
data.WoodDeckSF = data.WoodDeckSF.apply(lambda x: int(x))

In [57]:
data.OpenPorchSF = data.OpenPorchSF.apply(lambda x: int(x))

In [59]:
data.EnclosedPorch = data.EnclosedPorch.apply(lambda x: int(x))

In [95]:
data.3SsnPorch = data.3SsnPorch.apply(lambda x: int(x))

SyntaxError: invalid syntax (<ipython-input-95-6190c3305f64>, line 1)

In [71]:
total = DataFrame(Data.columns=['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch'])
total.sum(axis=0) 

SyntaxError: keyword can't be an expression (<ipython-input-71-b24431676d37>, line 1)

In [92]:
data.Fence = data.Fence.replace('NaN','0',regex = True)

In [88]:
data.Fence = data.Fence.replace('GdWo','1',regex = True)

In [89]:
data.Fence = data.Fence.replace('GdPrv','2',regex = True)

In [90]:
data.Fence = data.Fence.replace('MnPrv','3',regex = True)

In [93]:
data.Fence.sample(10)

1146    NaN
650     NaN
437     NaN
413     NaN
935     NaN
1305    NaN
17      NaN
578     NaN
1356      1
322       3
Name: Fence, dtype: object

In [94]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [96]:
data.MiscVal = data.MiscVal.apply(lambda x: int(x))

In [99]:
pd.to_datatime[data.MoSold + data.YrSold]

AttributeError: module 'pandas' has no attribute 'to_datatime'

In [100]:
data.SaleType = data.SaleType.replace('New','1',regex = True)

In [101]:
data.SaleType = data.SaleType.replace('WD','2',regex = True)

In [102]:
data.SaleType = data.SaleType.replace('COD','3',regex = True)

In [103]:
# data.SaleType.sample(10)

507     1
81      2
621     2
1363    1
628     2
78      2
1294    2
926     2
183     2
1028    2
Name: SaleType, dtype: object

In [107]:
data.SaleCondition = data.SaleCondition.replace('Abnormal','1',regex = True)

In [108]:
data.SaleCondition = data.SaleCondition.replace('Normal','2',regex = True)

In [109]:
data.SaleCondition = data.SaleCondition.replace('Partial','3',regex = True)

In [110]:
data.SaleCondition = data.SaleCondition.replace('Alloca','4',regex = True)

In [112]:
# data.SaleCondition.sample()

913    2
Name: SaleCondition, dtype: object

In [113]:
data.SalePrice = data.SalePrice.apply(lambda x: int(x))